In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy as sp
from hydromet.io import read_predictors, read_predictands
from hydromet.system import get_model_config, get_model_dir
%matplotlib inline
from phildb.database import PhilDB
db = PhilDB('hm_tsdb')

/Users/amacdon/anaconda/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
config = get_model_config('project', 'gr4j')

In [3]:
config['calibration']['end_date']

datetime.date(2008, 12, 1)

In [4]:
warmup_predictors = read_predictors(db, config, '410730', 'warmup')

In [5]:
predictors = read_predictors(db, config, '410730', 'calibration')

In [6]:
predictands = read_predictands(db, config, '410730', 'calibration')

In [7]:
from hydromet.models.gr4j_model import GR4J

In [8]:
m = GR4J()

In [10]:
#m.X1[0], m.X2[0], m.X3[0], m.X3[0]
params = (1139.9784914844802,
 -0.50826570328096576,
 87.051727183614759,
 87.051727183614759)

In [11]:
m.X1, m.X2, m.X3, m.X4 = params

In [12]:
fc_date = '2010-06-01'

In [17]:
for fc_date in pd.date_range('2010-01-01', '2011-02-01', freq='MS'):
    warmup_data = read_predictors(db, config, '410730', idx = slice('2009-01-01', '2009-12-01'))
    p = read_predictors(db, config, '410730', idx = slice(fc_date, fc_date))
    print(np.array(m.forecast(fc_date, warmup_data, p)) * 148 - read_predictands(db, config, '410730', idx = slice(fc_date, fc_date))['Q'].values[0])

[ 442.16748596]
[ 463.15247285]
[ 449.46828544]
[ 427.87268726]
[ 442.51068956]
[ 339.39069094]
[ 389.84379137]
[ 368.15368339]
[ 285.49668604]
[ 349.25768519]
[ 14.38168366]
[-218.47039558]
[ 348.35967947]
[ 366.700683]


In [9]:
%time m.calibrate(warmup_predictors, predictors, predictands)

differential_evolution step 1: f(x)= 0.368061
differential_evolution step 2: f(x)= 0.350424
differential_evolution step 3: f(x)= 0.284236
differential_evolution step 4: f(x)= 0.252971
differential_evolution step 5: f(x)= 0.252971
differential_evolution step 6: f(x)= 0.252971
differential_evolution step 7: f(x)= 0.252971
differential_evolution step 8: f(x)= 0.252971
differential_evolution step 9: f(x)= 0.252902
differential_evolution step 10: f(x)= 0.252902
differential_evolution step 11: f(x)= 0.251244
differential_evolution step 12: f(x)= 0.251244
differential_evolution step 13: f(x)= 0.251173
differential_evolution step 14: f(x)= 0.251173
differential_evolution step 15: f(x)= 0.251159
CPU times: user 3min, sys: 211 ms, total: 3min
Wall time: 3min


 success: True
     jac: array([ -3.89688282e-06,  -1.95732319e-05,  -1.99840144e-07,
        -1.84741111e-05])
     nit: 15
     fun: 0.25095223475137607
 message: 'Optimization terminated successfully.'
       x: array([  1.13997849e+03,  -5.08265703e-01,   8.70517272e+01,
         1.21582547e+00])
    nfev: 1140